fonte: https://data.world/uci/online-retail

In [2]:
import pandas as pd
import numpy as np

In [3]:
tb_retail = pd.read_excel('Online Retail.xlsx')

In [4]:
tb_retail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [5]:
(tb_retail["CustomerID"].value_counts()  > 50).sum()

1933

In [6]:
tb_client = pd.DataFrame({'num_pedidos' : tb_retail["CustomerID"].value_counts()})
tb_client['cum_rep'] = np.cumsum(tb_client['num_pedidos'])/sum(tb_client['num_pedidos'])
client_list = tb_client[tb_client['cum_rep'] < 0.95].index

In [7]:
mask_rel = tb_retail['CustomerID'].isin(client_list)
tb_relevant_retail = tb_retail[mask_rel].copy()
tb_relevant_retail.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 386465 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    386465 non-null  object        
 1   StockCode    386465 non-null  object        
 2   Description  386465 non-null  object        
 3   Quantity     386465 non-null  int64         
 4   InvoiceDate  386465 non-null  datetime64[ns]
 5   UnitPrice    386465 non-null  float64       
 6   CustomerID   386465 non-null  float64       
 7   Country      386465 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 26.5+ MB


In [13]:
tb_prod = pd.DataFrame({'num_pedidos' : tb_relevant_retail["StockCode"].value_counts()})
tb_prod['cum_rep'] = np.cumsum(tb_prod['num_pedidos'])/sum(tb_prod['num_pedidos'])
product_list = tb_prod[tb_prod['cum_rep'] < 0.70].index

In [14]:
mask_rel = tb_relevant_retail['StockCode'].isin(product_list)
tb_relevant_retail = tb_relevant_retail[mask_rel].copy()
tb_relevant_retail.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 256927 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    256927 non-null  object        
 1   StockCode    256927 non-null  object        
 2   Description  256927 non-null  object        
 3   Quantity     256927 non-null  int64         
 4   InvoiceDate  256927 non-null  datetime64[ns]
 5   UnitPrice    256927 non-null  float64       
 6   CustomerID   256927 non-null  float64       
 7   Country      256927 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 17.6+ MB


In [15]:
tb_relevant_retail_agg = tb_relevant_retail.groupby(['CustomerID', 'StockCode'])['Country'].count().reset_index()

In [16]:
tb_client_product = pd.crosstab(
    index = tb_relevant_retail_agg['CustomerID'], 
    columns = tb_relevant_retail_agg['StockCode']).reset_index()

In [17]:
tb_client_product.shape

(2783, 755)

In [18]:
tb_client_product.to_csv('data/tb_client_product.csv', index = False)
tb_relevant_retail.to_csv('data/tb_retail.csv', index = False)

In [2]:
import pandas as pd

In [5]:
tb_house = pd.read_csv('data/household_power_consumption.txt', 
                       sep = ';', decimal = '.', na_values='?')
tb_house['date_time_str'] = tb_house['Date'] + " " + tb_house['Time']
tb_house.columns = [x.lower() for x in tb_house.columns]
tb_house = tb_house.dropna()
tb_house_samp = tb_house.sample(10000)
tb_house_samp['date_time'] = pd.to_datetime(tb_house_samp['date_time_str'])

In [7]:
tb_house_samp.columns

Index(['date', 'time', 'global_active_power', 'global_reactive_power',
       'voltage', 'global_intensity', 'sub_metering_1', 'sub_metering_2',
       'sub_metering_3', 'date_time_str', 'date_time'],
      dtype='object')

In [8]:
imp_vars = ['global_active_power', 'global_reactive_power', 'voltage', 'global_intensity', 'sub_metering_1', 'sub_metering_2', 'sub_metering_3', 'date_time']
tb_house_samp[imp_vars].to_csv('data/tb_household_electricity.csv', index = False)